# Tools 2 Final Project

Jen Lee
Connie Chang

In [ ]:
# !pip install geopandas
# !pip install geopandas --upgrade 

In [ ]:
import pandas as pd
import geopandas as gpd
import re
import numpy as np

## Data Introduction

|Date Compiled |County/City|File Nale   |Description |
|:-------------|:---------:|:----------:|-----------:|
|29 March, 2021|Denver     |listings.csv|Summary information and metrics for listings in Denver (good for visualisations).|
|29 March, 2021|Denver     |calendar.csv| Detailed Calendar Data for listings in Denver|
|29 March, 2021|Denver     |reviews.csv| Summary Review data and Listing ID (to facilitate time based analytics and visualisations linked to a listing).|
|29 March, 2021|Denver     |neighbourhoods.csv| Neighbourhood list for geo filter. Sourced from city or open source GIS files.|
|29 March, 2021|Denver     |neighbourhoods.geojson| GeoJSON file of neighbourhoods of the city.|


Import each of the csv files. Not sure which ones we'll use yet, but might as well import all of them.



In [ ]:
# listings.csv
listings = pd.read_csv("./data/denver_03_29_2021/listings.csv")
listings.head()

In [ ]:
# calendar.csv
calendar = pd.read_csv("./data/denver_03_29_2021/calendar.csv")
calendar.head()

In [ ]:
# reviews.csv
reviews = pd.read_csv("./data/denver_03_29_2021/reviews.csv")
reviews.head()

In [ ]:
# neighbourhoods.csv
neighborhoods = pd.read_csv("./data/denver_03_29_2021/neighbourhoods.csv")
neighborhoods.head()

In [ ]:
neighborhoods_geojson = gpd.read_file("./data/denver_03_29_2021/neighbourhoods.geojson")
neighborhoods_geojson.head()

## Data Cleaning

In [ ]:
# listings.csv

# remove unneeded columns
listings = listings.drop(["picture_url", "host_url", "host_thumbnail_url", "host_picture_url", "license"], axis=1)

# TODO: replace all empty columns with NaN

# change columns with 't' & 'f' to booleans
def convert_to_bool(x):
    if x == "t":
        return True
    elif x == "f":
        return False
    else:
        return np.NaN

listings["host_is_superhost"] = listings["host_is_superhost"].apply(lambda x: convert_to_bool(x))
listings["host_identity_verified"] = listings["host_identity_verified"].apply(lambda x: convert_to_bool(x))
listings["has_availability"] = listings["has_availability"].apply(lambda x: convert_to_bool(x))
listings["instant_bookable"] = listings["instant_bookable"].apply(lambda x: convert_to_bool(x))

# Clean up the "bathrooms_text" column to make two new columns - number of bathrooms, and a boolean column to indicate if bthrooms are private
def get_bathrooms_num(x):
    if type(x) != str:
        return 0.0
    elif x == "Shared half-bath" or x == "Half-bath":
        return 0.5
    elif "shared bath" in x:
        return float(x.strip("s").strip("shared bath"))
    elif "private bath" in x:
        return float(x.strip("s").strip("private bath"))
    elif "bath" in x:
        return float(x.strip("s").strip("bath"))
    else:
        return 0.0
    
def get_is_private_bathroom(x):
    if type(x) != str:
        return np.NaN
    elif "shared" in x:
        return False
    else:
        return True

listings["bathrooms"] = listings["bathrooms_text"].apply(lambda x: get_bathrooms_num(x))
listings["private_bathroom"] = listings["bathrooms_text"].apply(lambda x: get_is_private_bathroom(x))
listings = listings.drop("bathrooms_text", axis=1)

# display head to show success
listings.head()


,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,private_bathroom
0,177,https://www.airbnb.com/rooms/177,20210329150448,2021-03-30,Tiny Home in the Heart of the City- ECO FRIENDLY,"160 sq ft + 80 sq ft loft for sleeping, Sleeps...","Quiet neighborhood next to park, creeks and bi...",615,Joe,2008-07-07,"Denver, Colorado, United States","I'm originally from Oklahoma, but have lived i...",within an hour,100%,100%,True,Virginia Village,2.0,2.0,"['email', 'phone', 'google', 'reviews', 'kba']",t,False,"Denver, Colorado, United States",Virginia Village,NaN,39.69551,-104.92489,Entire guesthouse,Entire home/apt,2,1.0,NaN,0.0,"[""Wifi"", ""Kitchen"", ""Carbon monoxide alarm"", ""...",$85.00,1,400,1,1,1125,1125,1.0,1125.0,NaN,True,0,28,52,319,2021-03-30,78,39,0,2016-09-04,2020-12-31,96.0,10.0,10.0,10.0,10.0,10.0,9.0,True,2,2,0,0,1.40,True
1,360,https://www.airbnb.com/rooms/360,20210329150448,2021-03-30,Sit in the Peaceful Garden of the Chickadee Co...,Enjoy the famous Colorado weather and unplug i...,The cottage is located in the center of Lower ...,666,Jennifer & Giovanni,2008-07-08,"Denver, Colorado, United States",We are artists and tinkerers.\r\n \r\nWe enjoy...,within an hour,100%,91%,True,Highland,2.0,2.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,True,"Denver, Colorado, United States",Highland,NaN,39.76758,-105.00316,Entire guesthouse,Entire home/apt,4,1.0,2.0,2.0,"[""Fire extinguisher"", ""Full kitchen"", ""TV"", ""W...",$134.00,3,29,2,3,29,29,3.0,29.0,NaN,True,6,12,30,30,2021-03-30,139,26,1,2018-08-13,2021-02-28,100.0,10.0,10.0,10.0,10.0,10.0,10.0,False,2,2,0,0,4.34,True
2,364,https://www.airbnb.com/rooms/364,20210329150448,2021-03-31,Lodo / RiNo LOFT via airport train,"Modern 1,000 square foot loft in the heart of ...","Ten brewpubs within walking distance, two grea...",783,Jason,2008-07-11,"Denver, Colorado, United States","I travel pretty frequently, I spend a lot of t...",NaN,NaN,NaN,False,Five Points,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,True,"Denver, Colorado, United States",Five Points,NaN,39.76672,-104.97906,Entire loft,Entire home/apt,3,1.5,1.0,1.0,"[""Wifi"", ""Breakfast"", ""Kitchen"", ""Carbon monox...",$179.00,185,365,185,185,365,365,185.0,365.0,NaN,True,28,58,88,363,2021-03-31,87,0,0,2009-05-18,2016-04-26,97.0,10.0,10.0,10.0,10.0,9.0,9.0,False,1,1,0,0,0.60,True
3,590,https://www.airbnb.com/rooms/590,20210329150448,2021-04-02,Comfortable - and a great value!,"Large guest room in my home, where I also live...",I love the diversity of my neighborhood and it...,933,Jill,2008-07-21,"Denver, Colorado, United States",I am friendly and I love meeting people from a...,within an hour,100%,83%,True,North Park Hill,2.0,2.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,True,"Denver, Colorado, United States",North Park H

### Fill in NA
1. "Bedroom" field with blank are the ones that either a studio, or an open space without separate room as a bedroom, so we fill it with 0.
2. "Beds" field with blank typically means that the listing host missed the field. The main point of airbnb is bed, we'd assume they have at least 1 bed, so fill in the blank ones with 1.
3. "Reviews_per_month" field with blank means there's no review at all. Thus, the "first_review" and "last_review" fields are both blank. We'd fill the "reviews_per_month" with 0.

In [ ]:
# FunctionTransformer for fill NA 
from sklearn.preprocessing import FunctionTransformer
def custom_transformation(df):
  df.bedrooms.fillna(0, inplace=True)
  df.beds.fillna(1, inplace=True)
  df.reviews_per_month.fillna(0, inplace=True)

ft = FunctionTransformer(func=custom_transformation, validate=False)
ft.fit_transform(listings)


pd.set_option('display.max_rows',None)
listings.isna().sum()

id                                                 0
listing_url                                        0
scrape_id                                          0
last_scraped                                       0
name                                               0
description                                       37
neighborhood_overview                            802
host_id                                            0
host_name                                        151
host_since                                       151
host_location                                    158
host_about                                      1364
host_response_time                               471
host_response_rate                               471
host_acceptance_rate                             384
host_is_superhost                                151
host_neighbourhood                               508
host_listings_count                              151
host_total_listings_count                     

### Price Field
'price' field is with type: object. It needs to remove '$' and ',' then convert to numeric.

In [ ]:
print(f"listing price initial type: {listings.price.dtype}")

# remove '$' and ','
listings['price'] = listings['price'].str.strip('$')
listings['price'] = listings['price'].str.replace(',', '')

# convert to numeric
listings['price'] = pd.to_numeric(listings['price'])

print(f"listing price after cleaning type: {listings.price.dtype}")

listings['price'].describe()

listing price initial type: object
listing price after cleaning type: float64


count    3394.000000
mean      157.709782
std       244.756065
min         0.000000
25%        79.000000
50%       115.000000
75%       175.000000
max      9999.000000
Name: price, dtype: float64